In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,classification_report


In [2]:
df = pd.read_csv('../YouTubeDatasets/creditcard.csv',index_col=0)
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
Time,,,,,,,,,,,,,,,,,,,,,
0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
from collections import Counter
Counter(df.Class)

Counter({0: 284315, 1: 492})

In [8]:
X = df.drop('Class',axis=1)
y  = df.Class

## Model


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold,train_test_split,GridSearchCV

In [10]:
np.arange(-2,3)


array([-2, -1,  0,  1,  2])

In [11]:
log_model = LogisticRegression()

## C is the penalty we are applying in the model
grid = {'C':10.0**np.arange(-2,3),'penalty':['l1','l2']}
cv = KFold(n_splits=5)


In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25)

In [13]:
clf = GridSearchCV(log_model,grid,cv=cv,n_jobs=-1,scoring='f1_macro')
clf.fit(X_train,y_train)

D:\Anaconda\envs\gpu\lib\site-packages\sklearn\model_selection\_search.py:925: UserWarning: One or more of the test scores are non-finite: [       nan 0.8376201         nan 0.85963921        nan 0.84817459
        nan 0.84598661        nan 0.85939058]
  category=UserWarning
D:\Anaconda\envs\gpu\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro')

In [14]:
from sklearn.metrics import confusion_matrix,classification_report

y_pred = clf.predict(X_test)
print(accuracy_score(y_test,y_pred))
print("")
print(confusion_matrix(y_test,y_pred),)
print("")
print(classification_report(y_test,y_pred))

0.9991011488441336

[[71035    15]
 [   49   103]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     71050
           1       0.87      0.68      0.76       152

    accuracy                           1.00     71202
   macro avg       0.94      0.84      0.88     71202
weighted avg       1.00      1.00      1.00     71202



## Random Forest

In [15]:
class_weight = ({0:1,1:100})

## 0th class with weight of 1 and 1th Class with weight of 100
from sklearn.ensemble import RandomForestClassifier

rf =RandomForestClassifier(class_weight=class_weight)
rf.fit(X_train,y_train)

RandomForestClassifier(class_weight={0: 1, 1: 100})

In [30]:
y_pred = rf.predict(X_test)
print(accuracy_score(y_test,y_pred))
print("")
print(confusion_matrix(y_test,y_pred),)
print("")
print(classification_report(y_test,y_pred))

0.999691019915171

[[71091     2]
 [   20    89]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     71093
           1       0.98      0.82      0.89       109

    accuracy                           1.00     71202
   macro avg       0.99      0.91      0.94     71202
weighted avg       1.00      1.00      1.00     71202



## Undersampling

In [31]:
from collections import Counter
Counter(y_train)


## clearly the data is imbalanced
## and the model is over fittin

Counter({0: 213222, 1: 383})

In [32]:
from imblearn.under_sampling import RandomUnderSampler
rs_under = RandomUnderSampler(random_state=42)
X_train_under,y_train_under = rs_under.fit_resample(X_train,y_train)

In [36]:
print(Counter(y_train_under))

Counter({0: 383, 1: 383})


In [37]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train_under,y_train_under)

RandomForestClassifier()

In [39]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("")
print(accuracy_score(y_test,y_pred))
print("")
print(classification_report(y_test,y_pred))
print("")

[[69079  2014]
 [   10    99]]

0.9715738321957248

              precision    recall  f1-score   support

           0       1.00      0.97      0.99     71093
           1       0.05      0.91      0.09       109

    accuracy                           0.97     71202
   macro avg       0.52      0.94      0.54     71202
weighted avg       1.00      0.97      0.98     71202




## Oversampling

In [40]:
from imblearn.over_sampling import RandomOverSampler
rs_over = RandomOverSampler(random_state=42,sampling_strategy='minority')
X_train_over,y_train_over = rs_over.fit_resample(X_train,y_train)

In [41]:
Counter(y_train_over)

Counter({0: 213222, 1: 213222})

In [42]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train_over,y_train_over)

RandomForestClassifier()

In [43]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("")
print(accuracy_score(y_test,y_pred))
print("")
print(classification_report(y_test,y_pred))
print("")

[[71089     4]
 [   20    89]]

0.9996629308165501

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     71093
           1       0.96      0.82      0.88       109

    accuracy                           1.00     71202
   macro avg       0.98      0.91      0.94     71202
weighted avg       1.00      1.00      1.00     71202


